# Hệ Thống Hỏi Đáp GraphRAG - Quy Trình Hoàn Chỉnh

Notebook này chứa quy trình hoàn chỉnh để xây dựng và sử dụng hệ thống hỏi đáp dựa trên GraphRAG.

## Tổng Quan

Notebook này bao gồm:
1. Thiết Lập và Cấu Hình
2. Lập Chỉ Mục - Xây Dựng Đồ Thị Tri Thức từ Tài Liệu
3. Truy Vấn - Đặt Câu Hỏi và Nhận Câu Trả Lời với Các Cải Tiến
4. Trực Quan Hóa - Hiển Thị Đồ Thị Tri Thức (Tùy Chọn)

## Yêu Cầu

- Python 3.10 trở lên
- Khóa API OpenAI được đặt trong biến môi trường: GRAPHRAG_API_KEY
- Tài liệu đầu vào trong thư mục `input/`
- Các phụ thuộc dự án đã được cài đặt (chạy `poetry install` nếu cần)

## Cách Chạy

1. Chạy tất cả các cell từ trên xuống dưới bằng "Run All" trong menu Cell
2. HOẶC chạy từng phần riêng lẻ:
   - Thiết lập lần đầu: Chạy Phần 1
   - Xây dựng đồ thị tri thức: Chạy Phần 2
   - Truy vấn tài liệu: Chạy Phần 3
   - Trực quan hóa đồ thị: Chạy Phần 4 (tùy chọn)

## Lưu Ý Quan Trọng

- Phần 2 (Lập Chỉ Mục) mất 10-30 phút tùy thuộc vào kích thước tài liệu
- Phần 2 yêu cầu tín dụng API OpenAI (tốn phí)
- Nếu bạn đã có dữ liệu đã lập chỉ mục, bạn có thể bỏ qua Phần 2
- Tất cả các cải tiến đã được tích hợp và bật theo mặc định


# Phần 1: Thiết Lập và Cấu Hình

Phần này kiểm tra môi trường của bạn, xác minh các phụ thuộc, và chuẩn bị hệ thống cho việc lập chỉ mục và truy vấn.


In [ ]:
# Kiểm tra môi trường và thiết lập
import os
import sys
import glob
from pathlib import Path

# Lấy thư mục gốc của dự án
PROJECT_ROOT = os.getcwd()
print(f"Thư mục gốc dự án: {PROJECT_ROOT}")

# Kiểm tra các file bắt buộc
required_files = ['settings.yaml', 'pyproject.toml']
missing_files = []
for file in required_files:
    if os.path.exists(file):
        print(f"Đã tìm thấy: {file}")
    else:
        print(f"Thiếu: {file}")
        missing_files.append(file)

if missing_files:
    print(f"\nCảnh báo: Thiếu các file bắt buộc: {missing_files}")

# Kiểm tra khóa API
api_key = os.environ.get("GRAPHRAG_API_KEY")
if api_key:
    print(f"\nĐã tìm thấy khóa API: {api_key[:10]}...")
else:
    print("\nLỖI: Biến môi trường GRAPHRAG_API_KEY chưa được đặt!")
    print("Vui lòng đặt bằng: export GRAPHRAG_API_KEY='your-key-here'")
    print("Hoặc trong Python: os.environ['GRAPHRAG_API_KEY'] = 'your-key-here'")

# Kiểm tra thư mục đầu vào
if os.path.exists('input'):
    input_files = [f for f in os.listdir('input') if os.path.isfile(os.path.join('input', f))]
    print(f"\nĐã tìm thấy thư mục đầu vào với {len(input_files)} file:")
    for f in input_files:
        file_path = os.path.join('input', f)
        file_size = os.path.getsize(file_path) / (1024 * 1024)  # MB
        print(f"  - {f} ({file_size:.2f} MB)")
else:
    print("\nCảnh báo: Không tìm thấy thư mục đầu vào!")
    print("Vui lòng tạo thư mục 'input/' và thêm các tài liệu của bạn (file .txt)")

# Kiểm tra các thư mục đầu ra hiện có
output_dirs = glob.glob("output/*/")
if output_dirs:
    latest_output = max(output_dirs, key=os.path.getctime)
    print(f"\nĐã tìm thấy kết quả lập chỉ mục hiện có: {latest_output}")
    print("Bạn có thể bỏ qua Phần 2 (Lập Chỉ Mục) nếu muốn sử dụng dữ liệu hiện có này.")
else:
    print("\nKhông tìm thấy kết quả lập chỉ mục hiện có.")
    print("Bạn cần chạy Phần 2 (Lập Chỉ Mục) trước.")

print("\n" + "="*80)
print("Kiểm tra thiết lập hoàn tất. Vui lòng xem lại các cảnh báo ở trên trước khi tiếp tục.")
print("="*80)


# Phần 2: Lập Chỉ Mục - Xây Dựng Đồ Thị Tri Thức

Phần này xây dựng đồ thị tri thức từ các tài liệu của bạn. Quá trình này:

1. Đọc tài liệu từ thư mục `input/`
2. Trích xuất các thực thể và mối quan hệ bằng LLM
3. Xây dựng đồ thị tri thức
4. Tạo embeddings cho tìm kiếm ngữ nghĩa
5. Lưu kết quả vào thư mục `output/`


In [ ]:
# Chạy pipeline lập chỉ mục
# Bỏ comment code bên dưới và chạy cell này để bắt đầu lập chỉ mục

import subprocess
import sys
import glob

def run_indexing(config_file="settings.yaml", verbose=True):
    """
    Chạy pipeline lập chỉ mục GraphRAG.
    
    Hàm này sẽ xử lý tất cả các tài liệu trong thư mục input/ và tạo
    đồ thị tri thức với các thực thể, mối quan hệ và embeddings.
    """
    print("Đang bắt đầu pipeline lập chỉ mục...")
    print(f"Sử dụng file cấu hình: {config_file}")
    print("Quá trình này có thể mất 10-30 phút tùy thuộc vào kích thước tài liệu...")
    print("-"*80)
    
    # Xây dựng lệnh
    cmd = [
        sys.executable,
        "-m", "graphrag.index",
        "--config", config_file,
    ]
    
    if verbose:
        cmd.append("--verbose")
    
    try:
        # Chạy lập chỉ mục
        result = subprocess.run(
            cmd,
            check=False,
            capture_output=False,  # Hiển thị output theo thời gian thực
            text=True
        )
        
        print("-"*80)
        if result.returncode == 0:
            print("Lập chỉ mục hoàn tất thành công!")
            # Tìm thư mục đầu ra
            output_dirs = glob.glob("output/*/")
            if output_dirs:
                latest = max(output_dirs, key=os.path.getctime)
                print(f"Đã lưu kết quả vào: {latest}")
                print(f"Vị trí artifacts: {latest}artifacts/")
            return True
        else:
            print(f"Lập chỉ mục thất bại với mã thoát: {result.returncode}")
            return False
            
    except Exception as e:
        print(f"Lỗi khi chạy lập chỉ mục: {e}")
        import traceback
        traceback.print_exc()
        return False

# BỎ COMMENT DÒNG BÊN DƯỚI ĐỂ CHẠY LẬP CHỈ MỤC
# run_indexing()

print("Để bắt đầu lập chỉ mục, bỏ comment dòng: run_indexing()")
print("Đảm bảo bạn có:")
print("  1. Tài liệu trong thư mục input/")
print("  2. Biến môi trường GRAPHRAG_API_KEY đã được đặt")
print("  3. Tín dụng API OpenAI có sẵn")


# Phần 3: Truy Vấn - Đặt Câu Hỏi và Nhận Câu Trả Lời

Phần này tải đồ thị tri thức đã lập chỉ mục và cho phép bạn đặt câu hỏi.

Các tính năng bao gồm:
- Trích dẫn nguồn nâng cao với các đoạn văn bản thực tế
- Xử lý fallback tự động (tìm kiếm cục bộ -> tìm kiếm toàn cục)
- Xử lý lỗi tốt hơn với thông báo thân thiện với người dùng
- Định dạng phản hồi với metadata

Bạn có thể đặt câu hỏi bằng tiếng Việt và nhận câu trả lời dựa trên các tài liệu đã lập chỉ mục của bạn.


In [ ]:
# Import các thư viện cần thiết
import pandas as pd
import tiktoken

from graphrag.query.context_builder.entity_extraction import EntityVectorStoreKey
from graphrag.query.indexer_adapters import (
    read_indexer_covariates,
    read_indexer_entities,
    read_indexer_relationships,
    read_indexer_reports,
    read_indexer_text_units,
)
from graphrag.query.input.loaders.dfs import (
    store_entity_semantic_embeddings,
)
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.embedding import OpenAIEmbedding
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.structured_search.local_search.mixed_context import (
    LocalSearchMixedContext,
)
from graphrag.query.structured_search.local_search.search import LocalSearch
from graphrag.query.structured_search.global_search.search import GlobalSearch
from graphrag.query.structured_search.global_search.community_context import (
    GlobalCommunityContext,
)
from graphrag.vector_stores.lancedb import LanceDBVectorStore

# Import các tiện ích cải tiến
sys.path.append('utils')
try:
    from qa_improvements import (
        format_sources_with_context,
        enhance_response_with_metadata,
        search_with_fallback,
        HybridSearch,
    )
    IMPROVEMENTS_AVAILABLE = True
    print("Đã tải các cải tiến thành công")
except ImportError:
    IMPROVEMENTS_AVAILABLE = False
    print("Cảnh báo: Các tiện ích cải tiến không khả dụng")
    print("Sử dụng chế độ tiêu chuẩn (các cải tiến sẽ bị bỏ qua)")

print("Tất cả các import đã thành công!")


In [ ]:
# Cấu hình - Tự động phát hiện thư mục đầu ra mới nhất hoặc đặt thủ công

output_dirs = sorted(glob.glob("output/*/"), key=os.path.getctime, reverse=True)
if output_dirs:
    INPUT_DIR = f"{output_dirs[0]}artifacts/"
    print(f"Sử dụng thư mục đầu ra: {output_dirs[0]}")
else:
    # Fallback - cập nhật nếu bạn có thư mục đầu ra cụ thể
    INPUT_DIR = "output/20240805-112918/artifacts/"
    print(f"Cảnh báo: Sử dụng thư mục mặc định: {INPUT_DIR}")
    print("Nếu điều này không đúng, hãy cập nhật INPUT_DIR trong cell ở trên")

LANCEDB_URI = f"{INPUT_DIR}/lancedb"

# Tên bảng (đầu ra tiêu chuẩn của GraphRAG)
COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"
RELATIONSHIP_TABLE = "create_final_relationships"
COVARIATE_TABLE = "create_final_covariates"
TEXT_UNIT_TABLE = "create_final_text_units"
COMMUNITY_LEVEL = 2

print(f"\nThư mục đầu vào: {INPUT_DIR}")
print(f"LanceDB URI: {LANCEDB_URI}")

# Xác minh thư mục tồn tại
if not os.path.exists(INPUT_DIR):
    print(f"\nLỖI: Thư mục đầu vào không tồn tại: {INPUT_DIR}")
    print("Vui lòng chạy Phần 2 (Lập Chỉ Mục) trước, hoặc cập nhật INPUT_DIR ở trên")
else:
    print("Đã xác minh thư mục đầu vào")


In [ ]:
# Tải các thực thể
print("Đang tải các thực thể...")

entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)

# Thiết lập embeddings mô tả thực thể trong LanceDB
description_embedding_store = LanceDBVectorStore(
    collection_name="entity_description_embeddings",
)
description_embedding_store.connect(db_uri=LANCEDB_URI)
store_entity_semantic_embeddings(
    entities=entities, vectorstore=description_embedding_store
)

print(f"Đã tải {len(entity_df)} thực thể")
print(f"Embeddings thực thể đã được lưu trong LanceDB")
entity_df.head()


In [ ]:
# Tải các mối quan hệ
print("Đang tải các mối quan hệ...")

relationship_df = pd.read_parquet(f"{INPUT_DIR}/{RELATIONSHIP_TABLE}.parquet")
relationships = read_indexer_relationships(relationship_df)

print(f"Đã tải {len(relationship_df)} mối quan hệ")
relationship_df.head()


In [ ]:
# Tải covariates (các khẳng định)
print("Đang tải covariates (các khẳng định)...")

covariate_df = pd.read_parquet(f"{INPUT_DIR}/{COVARIATE_TABLE}.parquet")
claims = read_indexer_covariates(covariate_df)
covariates = {"claims": claims}

print(f"Đã tải {len(claims)} khẳng định")


In [ ]:
# Tải báo cáo cộng đồng
print("Đang tải báo cáo cộng đồng...")

report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)

print(f"Đã tải {len(report_df)} báo cáo cộng đồng")
report_df.head()


In [ ]:
# Tải các đơn vị văn bản
print("Đang tải các đơn vị văn bản...")

text_unit_df = pd.read_parquet(f"{INPUT_DIR}/{TEXT_UNIT_TABLE}.parquet")
text_units = read_indexer_text_units(text_unit_df)

print(f"Đã tải {len(text_unit_df)} đơn vị văn bản")
text_unit_df.head()


In [ ]:
# Thiết lập các mô hình LLM và embedding
print("Đang thiết lập các mô hình LLM và embedding...")

api_key = os.environ.get("GRAPHRAG_API_KEY")
if not api_key:
    raise ValueError("Biến môi trường GRAPHRAG_API_KEY chưa được đặt!")

# Cấu hình mô hình
llm_model = "gpt-3.5-turbo-0125"  # Có thể nâng cấp lên "gpt-4-turbo-preview" để có chất lượng tốt hơn
embedding_model = "text-embedding-3-small"

# Khởi tạo LLM
llm = ChatOpenAI(
    api_key=api_key,
    model=llm_model,
    api_type=OpenaiApiType.OpenAI,
    max_retries=20,
)

# Khởi tạo bộ mã hóa token
token_encoder = tiktoken.get_encoding("cl100k_base")

# Khởi tạo mô hình embedding
text_embedder = OpenAIEmbedding(
    api_key=api_key,
    api_base=None,
    api_type=OpenaiApiType.OpenAI,
    model=embedding_model,
    deployment_name=embedding_model,
    max_retries=20,
)

print(f"Mô hình LLM: {llm_model}")
print(f"Mô hình embedding: {embedding_model}")
print("Đã khởi tạo các mô hình thành công")


In [ ]:
# Tạo bộ xây dựng ngữ cảnh tìm kiếm cục bộ
print("Đang tạo bộ xây dựng ngữ cảnh tìm kiếm cục bộ...")

local_context_builder = LocalSearchMixedContext(
    community_reports=reports,
    text_units=text_units,
    entities=entities,
    relationships=relationships,
    covariates=covariates,
    entity_text_embeddings=description_embedding_store,
    embedding_vectorstore_key=EntityVectorStoreKey.ID,
    text_embedder=text_embedder,
    token_encoder=token_encoder,
)

# Tạo bộ xây dựng ngữ cảnh tìm kiếm toàn cục (để fallback)
global_context_builder = GlobalCommunityContext(
    community_reports=reports,
    entities=entities,
    token_encoder=token_encoder,
)

print("Đã tạo các bộ xây dựng ngữ cảnh thành công")


In [ ]:
# Cấu hình tham số tìm kiếm
print("Đang cấu hình tham số tìm kiếm...")

# Tham số tìm kiếm cục bộ
local_context_params = {
    "text_unit_prop": 0.5,
    "community_prop": 0.1,
    "conversation_history_max_turns": 5,
    "conversation_history_user_turns_only": True,
    "top_k_mapped_entities": 10,
    "top_k_relationships": 10,
    "include_entity_rank": True,
    "include_relationship_weight": True,
    "include_community_rank": False,
    "return_candidate_context": False,
    "embedding_vectorstore_key": EntityVectorStoreKey.ID,
    "max_tokens": 12_000,
}

# Tham số LLM
llm_params = {
    "max_tokens": 2_000,
    "temperature": 0.0,
}

print("Đã cấu hình tham số tìm kiếm")


In [ ]:
# Tạo các công cụ tìm kiếm
print("Đang tạo các công cụ tìm kiếm...")

# Công cụ tìm kiếm cục bộ
local_search = LocalSearch(
    llm=llm,
    context_builder=local_context_builder,
    token_encoder=token_encoder,
    llm_params=llm_params,
    context_builder_params=local_context_params,
    response_type="multiple paragraphs",
)

# Công cụ tìm kiếm toàn cục (để fallback)
global_search = GlobalSearch(
    llm=llm,
    context_builder=global_context_builder,
    token_encoder=token_encoder,
    llm_params=llm_params,
    response_type="multiple paragraphs",
)

# Tạo tìm kiếm lai với fallback (nếu có cải tiến)
if IMPROVEMENTS_AVAILABLE:
    hybrid_search = HybridSearch(local_search, global_search, llm)
    print("Đã tạo công cụ tìm kiếm lai (với cải tiến)")
else:
    hybrid_search = local_search  # Fallback chỉ dùng tìm kiếm cục bộ
    print("Đã tạo công cụ tìm kiếm cục bộ (chế độ tiêu chuẩn)")

print("Các công cụ tìm kiếm đã sẵn sàng!")


## Đặt Câu Hỏi

Bây giờ bạn có thể đặt câu hỏi bằng tiếng Việt. Hệ thống sẽ:
1. Tìm kiếm đồ thị tri thức để tìm thông tin liên quan
2. Tạo câu trả lời dựa trên các tài liệu đã lập chỉ mục
3. Cung cấp trích dẫn nguồn cho biết thông tin đến từ đâu
4. Tự động fallback sang tìm kiếm toàn cục nếu tìm kiếm cục bộ thất bại (khi các cải tiến được bật)


In [ ]:
# Ví dụ truy vấn - sửa đổi câu hỏi bên dưới để đặt câu hỏi của riêng bạn
question = "Thời gian giao dịch trái phiếu chính phủ tại Sở Giao dịch Chứng khoán Hà Nội như thế nào?"

print("="*80)
print("CÂU HỎI:")
print(question)
print("="*80)
print("\nĐang tìm kiếm và tạo câu trả lời...\n")

try:
    # Sử dụng tìm kiếm lai với fallback nếu có, nếu không thì dùng tìm kiếm cục bộ
    if IMPROVEMENTS_AVAILABLE:
        result = await search_with_fallback(
            question,
            local_search,
            global_search
        )
        
        # Nâng cao phản hồi với định dạng và nguồn tốt hơn
        enhanced_response = enhance_response_with_metadata(
            result.response,
            result.context_data,
        )
        
        print("CÂU TRẢ LỜI:")
        print("-"*80)
        print(enhanced_response)
        print("-"*80)
    else:
        # Chế độ tiêu chuẩn không có cải tiến
        result = await local_search.asearch(question)
        print("CÂU TRẢ LỜI:")
        print("-"*80)
        print(result.response)
        print("-"*80)
        
        # Hiển thị thông tin nguồn cơ bản nếu có
        if hasattr(result, 'context_data') and 'sources' in result.context_data:
            sources = result.context_data['sources']
            print(f"\nNguồn: {len(sources)} đơn vị văn bản được tham chiếu")
        
except Exception as e:
    print(f"Lỗi: {str(e)}")
    import traceback
    traceback.print_exc()


In [ ]:
# Cell truy vấn tương tác - Chạy cell này để đặt nhiều câu hỏi
# Bạn có thể sửa đổi cell này để đặt các câu hỏi khác nhau

def ask_question(q):
    """Hàm hỗ trợ để đặt câu hỏi và nhận câu trả lời được nâng cao."""
    print("="*80)
    print(f"CÂU HỎI: {q}")
    print("="*80)
    print("\nĐang tìm kiếm...\n")
    
    try:
        if IMPROVEMENTS_AVAILABLE:
            result = await search_with_fallback(q, local_search, global_search)
            enhanced_response = enhance_response_with_metadata(
                result.response,
                result.context_data,
            )
            print("CÂU TRẢ LỜI:")
            print("-"*80)
            print(enhanced_response)
            print("-"*80)
        else:
            result = await local_search.asearch(q)
            print("CÂU TRẢ LỜI:")
            print("-"*80)
            print(result.response)
            print("-"*80)
        
        return result
        
    except Exception as e:
        print(f"Lỗi: {str(e)}")
        return None

# Ví dụ câu hỏi - bỏ comment và sửa đổi theo nhu cầu
# result1 = await ask_question("Phí giao dịch chứng khoán tại MBS là bao nhiêu?")
# result2 = await ask_question("MBS cung cấp những dịch vụ gì?")
# result3 = await ask_question("Có thể giao dịch cổ phiếu lô lẻ tại MBS không?")

print("Để đặt câu hỏi, sử dụng: result = await ask_question('câu hỏi của bạn ở đây')")


# Phần 4: Trực Quan Hóa - Hiển Thị Đồ Thị Neo4j

Phần này giúp bạn trực quan hóa đồ thị tri thức bằng Neo4j.

Lưu ý: Trực quan hóa Neo4j là tùy chọn và yêu cầu:
- Neo4j đã được cài đặt trên hệ thống của bạn
- Máy chủ Neo4j đang chạy

Nếu bạn không có Neo4j hoặc không muốn trực quan hóa, bạn có thể bỏ qua phần này.


In [ ]:
# Các hàm thiết lập và quản lý Neo4j

import subprocess
from pathlib import Path

def find_neo4j_binary():
    """Tìm file nhị phân Neo4j ở các vị trí phổ biến."""
    # Kiểm tra thư mục hiện tại trước
    local_neo4j = Path("neo4j-community-5.21.2/bin/neo4j")
    if local_neo4j.exists():
        return str(local_neo4j.parent)
    
    # Kiểm tra các đường dẫn cài đặt phổ biến
    common_paths = [
        "/usr/local/neo4j/bin",
        "/opt/neo4j/bin",
        "~/.neo4j/bin",
    ]
    
    for path in common_paths:
        expanded = Path(path).expanduser()
        if (expanded / "neo4j").exists():
            return str(expanded)
    
    return None

NEO4J_BIN_PATH = find_neo4j_binary()

if NEO4J_BIN_PATH:
    print(f"Đã tìm thấy Neo4j tại: {NEO4J_BIN_PATH}")
    
    def start_neo4j():
        """Khởi động máy chủ Neo4j."""
        start_command = [f'{NEO4J_BIN_PATH}/neo4j', 'start']
        result = subprocess.run(start_command, capture_output=True, text=True)
        print(result.stdout)
        if result.returncode == 0:
            print("Đã khởi động Neo4j thành công")
        else:
            print(f"Lỗi khi khởi động Neo4j: {result.stderr}")
    
    def stop_neo4j():
        """Dừng máy chủ Neo4j."""
        stop_command = [f'{NEO4J_BIN_PATH}/neo4j', 'stop']
        result = subprocess.run(stop_command, capture_output=True, text=True)
        print(result.stdout)
        if result.returncode == 0:
            print("Đã dừng Neo4j thành công")
        else:
            print(f"Lỗi khi dừng Neo4j: {result.stderr}")
    
    def check_neo4j_status():
        """Kiểm tra trạng thái máy chủ Neo4j."""
        status_command = [f'{NEO4J_BIN_PATH}/neo4j', 'status']
        result = subprocess.run(status_command, capture_output=True, text=True)
        print("Trạng thái Neo4j:")
        print(result.stdout)
        if result.stderr:
            print(result.stderr)
    
    print("\nCác hàm Neo4j có sẵn:")
    print("  - start_neo4j()     : Khởi động máy chủ Neo4j")
    print("  - stop_neo4j()      : Dừng máy chủ Neo4j")
    print("  - check_neo4j_status() : Kiểm tra trạng thái máy chủ")
    print("\nGiao diện web Neo4j: http://localhost:7474")
    print("Thông tin đăng nhập mặc định: neo4j / neo4j (thay đổi khi đăng nhập lần đầu)")
    
else:
    print("Không tìm thấy Neo4j ở các vị trí phổ biến.")
    print("Nếu bạn đã cài đặt Neo4j, hãy cập nhật NEO4J_BIN_PATH trong cell ở trên.")
    print("Bạn có thể bỏ qua trực quan hóa nếu không cần.")


In [ ]:
# Khởi động Neo4j (bỏ comment để chạy)
# check_neo4j_status()

# Để khởi động Neo4j:
# start_neo4j()

# Để dừng Neo4j:
# stop_neo4j()

print("Để quản lý Neo4j, bỏ comment các lệnh ở trên hoặc sử dụng:")
print("  - start_neo4j()")
print("  - stop_neo4j()")
print("  - check_neo4j_status()")
